In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Seg3D/KiTS2019/kits19

/content/drive/MyDrive/Seg3D/KiTS2019/kits19


In [3]:
# !pip install git+https://github.com/shijianjian/EfficientNet-PyTorch-3D
# !pip install torchio


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  

Wed May 10 16:12:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import numpy as np

In [6]:
import glob

preds = glob.glob("/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/predictions/*")
len(preds)


90

In [7]:
random_choice = np.random.choice(preds)
random_msk = np.load(random_choice)

print("random_choice:", random_choice)
print("value:", np.unique(random_msk))
print("shape:", random_msk.shape)

random_choice: /content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/predictions/prediction_00248.nii.gz.npy
value: [0 1 2]
shape: (80, 160, 160)


**Prepare test data**

In [8]:
from segment.inference.prepare_test_data import prepare_test_data

In [9]:
# prepare_test_data(
#         data_path = "/content/drive/MyDrive/Seg3D/KiTS2019/kits19/data/",
#         config_path = "/content/drive/MyDrive/Seg3D/KiTS2019/kits19/configs/preprocess_pipeline.yaml",
#         vol_path = "/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/test_data/vol",
#         out_path = "/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/test_data.json"
# )


**Inference**

In [10]:
from segment.inference.infer_pipeline import inference

In [11]:
inference(
        data_path="/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/test_data.json", 
        out_path="/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/predictions",
        pred_path="/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/predictions.json",
        model_name_or_path="UnetRes_v2",
        config_path="/content/drive/MyDrive/Seg3D/KiTS2019/kits19/configs/preprocess_pipeline.yaml",
        pretrained_path="/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/models/best_model_2703_softmax.pt",
        freeze_feature=False,  
        num_classes=3,
        act_func="softmax"
)


<All keys matched successfully>
pretrained_path: /content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/models/best_model_2703_softmax.pt
num param: 91734833


inference :   0%|          | 0/90 [00:00<?, ?it/s]

test data shape: torch.Size([1, 1, 80, 160, 160])


0it [00:00, ?it/s]

postprocessing :   0%|          | 0/90 [00:00<?, ?it/s]

INFO:segment.utils.file_utils:The number of test dataset is 90
INFO:segment.utils.file_utils:Saved test dataset at /content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/predictions
